In [ ]:
using System;
using System.Collections.Concurrent;
using System.Collections.Generic;
using System.Linq;
using System.Threading;

public class LimitedConcurrentQueue<T>
{
    private readonly ConcurrentQueue<T> _queue = new ConcurrentQueue<T>();
    private readonly object _lock = new object();
    private readonly Queue<DateTime> _timeStamps = new Queue<DateTime>();

    private const int MaxItemsPerMinute = 4;
    private const int MaxItemsPerDay = 500;
    private const int Minute = 60;
    private const int Day = 24 * 60 * 60;

    public bool TryEnqueue(T item)
    {
        lock (_lock)
        {
            CleanUpOldTimestamps();

            if (_timeStamps.Count >= MaxItemsPerDay || 
                _timeStamps.Count(t => (DateTime.Now - t).TotalSeconds < Minute) >= MaxItemsPerMinute)
            {
                return false;
            }

            _queue.Enqueue(item);
            _timeStamps.Enqueue(DateTime.Now);
            return true;
        }
    }

    public bool TryDequeue(out T result)
    {
        return _queue.TryDequeue(out result);
    }

    private void CleanUpOldTimestamps()
    {
        while (_timeStamps.Count > 0 && (DateTime.Now - _timeStamps.Peek()).TotalSeconds > Day)
        {
            _timeStamps.Dequeue();
        }
    }
}


In [ ]:
using System;
using System.Net.Http;
using System.Net.Http.Headers;
using System.Threading.Tasks;
using Newtonsoft.Json.Linq;

public class VirusTotalAPI
{
    private readonly string _apiKey;
    private readonly HttpClient _httpClient;

    public VirusTotalAPI(string apiKey)
    {
        _apiKey = apiKey;
        _httpClient = new HttpClient();
    }

    public async Task<string> ScanFileAsync(string filePath)
    {
        var request = new HttpRequestMessage(HttpMethod.Post, "https://www.virustotal.com/api/v3/files");
        request.Headers.Add("x-apikey", _apiKey);

        var content = new MultipartFormDataContent();
        var fileContent = new ByteArrayContent(System.IO.File.ReadAllBytes(filePath));
        fileContent.Headers.ContentType = MediaTypeHeaderValue.Parse("application/octet-stream");
        content.Add(fileContent, "file", filePath);
        request.Content = content;


        var response = await _httpClient.SendAsync(request);
        response.EnsureSuccessStatusCode();
        var responseBody = await response.Content.ReadAsStringAsync();

        return responseBody;
    }
    
    public async Task<string> GetFileInfoAsync(string fileId)
{
    var request = new HttpRequestMessage(HttpMethod.Get, $"https://www.virustotal.com/api/v3/files/{fileId}");
    request.Headers.Add("x-apikey", _apiKey);

    var response = await _httpClient.SendAsync(request);
    response.EnsureSuccessStatusCode();
    var responseBody = await response.Content.ReadAsStringAsync();

    return responseBody;
}

}
